In [ ]:
# default_exp data.core

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Data

> This model contains all the necessary functionality for managing data. @Nathan


In [4]:
# export
import gdown
import icodegen
import io
import logging
import random
import re
import requests
import zipfile

import pandas as pd
import tensorflow as tf

# from datasets import load_dataset
from icodegen.data.transforms import java_comment_remover, transform_df
from pathlib import Path
from sklearn.model_selection import train_test_split
from subprocess import CalledProcessError, check_output
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors
from typing import Dict, Optional

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [5]:
# export
URLs = {
    "bigclonebenchmark": "https://drive.google.com/uc?id=1-AergjO9SGUDI3v8cj6UYisNZ1aJzxOx",
    "bigclonebenchmark_lg": "https://drive.google.com/uc?id=1-4LPiiKGR5Zmg-TLqZEkRbRIdg7UlJQb",
    "bigclonebenchmark_sm": "https://drive.google.com/uc?id=1FCq0lSs4oqc3jpSoucsHlRqjmbVwdRQ9",
    "bug_fix_pairs": "https://drive.google.com/uc?id=1XEhnsQ3Uy6SnFz349I0Iu9lz4ggAaiQp",
    "codesearchnet_java": "https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/java.zip",
}

In [6]:
# export
def _download_data(out_path):
    """
    Function for downloading all the data to reproduce our study.
    """
    # Download bigclonebenchmark_lg and bigclonebenchmark_sm
    logging.info("Downloading BigCloneBenchmark datasets.")
    bigclone_path = out_path / "bigclonebenchmark"
    bigclone_path.mkdir(parents=True, exist_ok=True)
    gdown.cached_download(
        URLs["bigclonebenchmark"],
        str(bigclone_path / "bigclonebenchmark.zip"),
        postprocess=gdown.extractall,
    )

    # Download Bug Fix Pairs
    logging.info("Downloading and extracting Bug Fix Pairs dataset.")
    bugfix_path = out_path / "bug_fix"
    bugfix_path.mkdir(parents=True, exist_ok=True)
    gdown.cached_download(
        URLs["bug_fix_pairs"],
        str(bugfix_path / "bug_fix_pairs.zip"),
        postprocess=gdown.extractall,
    )
    with zipfile.ZipFile(
        str(bugfix_path / "datasets" / "50-100" / "source_code.zip"), "r"
    ) as zip_ref:
        zip_ref.extractall(bugfix_path)

    # from https://stackoverflow.com/a/14260592/5768407 by users
    # yoavram (https://stackoverflow.com/users/1063612/yoavram) and
    # kamran kausar (https://stackoverflow.com/users/3486460/kamran-kausar)
    logging.info("Downloading and extracting CodeSearchNet Challenge dataset.")
    codesearchnet_path = out_path / "codesearchnet"
    if not codesearchnet_path.exists():
        codesearchnet_path.mkdir(parents=True, exist_ok=True)
        r = requests.get(URLs["codesearchnet_java"])
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall(codesearchnet_path / "codesearchnet_java")
    else:
        logging.info(f"File exists: {codesearchnet_path}")

In [ ]:
data_path = Path("/home/jovyan/work/dvc-icodegen/")
_download_data(data_path)

INFO:root:Downloading and extracting Bug Fix Pairs dataset.
Cached Downloading: /home/jovyan/work/dvc-icodegen/bug_fix/bug_fix_pairs.zip
Downloading...
From: https://drive.google.com/uc?id=1XEhnsQ3Uy6SnFz349I0Iu9lz4ggAaiQp
To: /home/jovyan/.cache/gdown/tmp29haq8_g/dl

0.00B [00:00, ?B/s]
1.05MB [00:00, 8.47MB/s]
3.67MB [00:00, 10.4MB/s]
6.29MB [00:00, 12.6MB/s]
7.86MB [00:00, 13.0MB/s]
10.5MB [00:00, 14.7MB/s]
12.6MB [00:00, 15.0MB/s]
15.7MB [00:00, 17.2MB/s]
17.8MB [00:00, 18.1MB/s]
20.4MB [00:01, 18.2MB/s]
23.6MB [00:01, 20.8MB/s]
26.2MB [00:01, 18.2MB/s]
28.3MB [00:01, 18.0MB/s]
30.4MB [00:01, 18.5MB/s]
32.5MB [00:01, 15.4MB/s]
35.1MB [00:01, 17.2MB/s]
38.3MB [00:02, 19.0MB/s]
40.9MB [00:02, 19.9MB/s]
43.5MB [00:02, 21.3MB/s]
46.1MB [00:02, 21.3MB/s]
48.8MB [00:02, 21.5MB/s]
51.4MB [00:02, 20.9MB/s]
54.5MB [00:02, 22.1MB/s]
57.1MB [00:02, 21.5MB/s]
59.8MB [00:02, 22.1MB/s]
62.5MB [00:03, 20.1MB/s]
64.6MB [00:03, 20.3MB/s]
67.2MB [00:03, 21.7MB/s]
69.7MB [00:03, 18.7MB/s]
71.8MB [00:

In [ ]:
# Commenting out to prevent CI/CD from wasting bandwith downloading
# data_path = Path("/tmp/data")
# _download_data(data_path)

# bigclone_path = data_path / "bigclonebenchmark"
# assert Path(
#     bigclone_path / "bigclone_types" / "bigclone_type_1.jsonl"
# ).exists()
# assert Path(
#     bigclone_path / "bigclone_types" / "bigclone_type_2.jsonl"
# ).exists()
# assert Path(
#     bigclone_path / "bigclone_types" / "bigclone_type_3.jsonl"
# ).exists()
# # assert Path(bigclone_path / "bigclonebenchmark_lg.csv").exists()
# # assert Path(bigclone_path / "bigclonebenchmark_sm.csv").exists()

# bugfix_path = data_path / "bug_fix"
# assert Path(bugfix_path / "bug_fix_pairs.zip").exists()
# assert Path(bugfix_path / "50-100/buggy").exists()
# assert Path(bugfix_path / "50-100/fixed").exists()

# codesearchnet_path = data_path / "codesearchnet"
# assert Path(codesearchnet_path / "codesearchnet_java").exists()

In [ ]:
# hide
df_fake = pd.DataFrame(["this is a test", "भारत test"], columns=["code"])
df_fake

,code
0,this is a test
1,भारत test


In [7]:
# export
def _isASCII(mthd: str) -> bool:
    """
    Check if the given method contains only ASCII characters. From https://stackoverflow.com/a/27084708/5768407.

    :param mthd: the method to verify contains only ASCII characters
    :returns: returns a boolean representing whether or not the given method contains only ASCII characters
    """
    try:
        mthd.encode(encoding="utf-8").decode("ascii")
    except UnicodeDecodeError:
        return False
    else:
        return True


def remove_non_ascii(df: pd.DataFrame, n: Optional[int] = None) -> pd.DataFrame:
    """
    Remove all methods that contain non-ascii characters from a given pandas dataframe, not in-place.

    :param df: the pandas dataframe containing each method to be beautified
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :returns: returns a new dataframe without methods that contain non-ascii characters
    """
    if n is None:
        n = len(df)

    df = df.iloc[:n].copy()
    df = df[df.code.apply(_isASCII)]

    return df

In [ ]:
NON_ASCII_DF = pd.DataFrame(["this is a test"], columns=["code"])
df_non_ascii = remove_non_ascii(df_fake)

assert (NON_ASCII_DF == df_non_ascii).all().all()

In [ ]:
# hide
df_fake = pd.DataFrame(
    [
        """public void setPipelines(java.util.Collection<Pipeline> pipelines) {
        if (pipelines == null) {
            this.pipelines = null;
            return;
        }

        this.pipelines = new com.amazonaws.internal.SdkInternalList<Pipeline>(pipelines);
    }
    """
    ],
    columns=["code"],
)
df_fake

,code
0,public void setPipelines(java.util.Collection<...


In [8]:
# export
def _beautify(mthd: str) -> str:
    """
    Beautifies a given method using uncrustify with the sun.cfg style, i.e., Oracle's style.

    :param mthd: the method to beautify
    :returns: returns a beautified version of the given method
    """
    # get path of icodegen
    icodegen_path = Path(icodegen.__path__[0])

    # create tmp file to store df contents for training tokenizer
    tmp_path = Path("/tmp")
    tmp_path.mkdir(parents=True, exist_ok=True)
    with open(tmp_path / "tmp.java", "w") as f:
        f.write(mthd)

    try:
        beaut_mthd = check_output(
            [
                icodegen_path / "uncrustify",
                "-c",
                icodegen_path / "sun.cfg",
                "-f",
                tmp_path / "tmp.java",
            ]
        ).decode("utf-8")
    except CalledProcessError as e:
        # Exception thrown when the method is malformed, i.e, it is missing a curly brace
        beaut_mthd = e.output.decode("utf-8")

    return beaut_mthd


def beautify_code(df: pd.DataFrame, n: Optional[int] = None) -> pd.DataFrame:
    """
    Beautify the methods in a pandas dataframe using uncrustify with the sun.cfg style, i.e., Oracle's style, not in-place.

    :param df: the pandas dataframe containing each method to be beautified
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :returns: returns a modified dataframe with the methods beautified
    """
    if n is None:
        n = len(df)

    df = df.iloc[:n].copy()
    df.code = df.code.apply(_beautify)

    return df

In [ ]:
BEAUT_MTHD = """public void setPipelines(java.util.Collection<Pipeline> pipelines) {
    if (pipelines == null) {
	this.pipelines = null;
	return;
    }
    this.pipelines = new com.amazonaws.internal.SdkInternalList<Pipeline>(
	pipelines);
}
"""

df_beaut = beautify_code(df_fake)

assert BEAUT_MTHD == df_beaut.code.values[0]

In [9]:
# export
# dicts of special tokens we are adding to the tokenizers so they do not get split

extra_tokens = {"<n>": "\n"}

# from https://docs.oracle.com/javase/tutorial/java/nutsandbolts/_keywords.html
java_reserved_tokens = {
    "<abstract>": "abstract",
    "<assert>": "assert",
    "<boolean>": "boolean",
    "<break>": "break",
    "<byte>": "byte",
    "<case>": "case",
    "<catch>": "catch",
    "<char>": "char",
    "<class>": "class",
    "<const>": "const",
    "<continue>": "continue",
    "<default>": "default",
    "<do>": "do",
    "<double>": "double",
    "<else>": "else",
    "<enum>": "enum",
    "<extends>": "extends",
    "<final>": "final",
    "<finally>": "finally",
    "<float>": "float",
    "<for>": "for",
    "<goto>": "goto",
    "<if>": "if",
    "<implements>": "implements",
    "<import>": "import",
    "<instanceof>": "instanceof",
    "<int>": "int",
    "<interface>": "interface",
    "<long>": "long",
    "<native>": "native",
    "<new>": "new",
    "<package>": "package",
    "<private>": "private",
    "<protected>": "protected",
    "<public>": "public",
    "<return>": "return",
    "<short>": "short",
    "<static>": "static",
    "<strictfp>": "strictfp",
    "<super>": "super",
    "<switch>": "switch",
    "<synchronized>": "synchronized",
    "<this>": "this",
    "<throw>": "throw",
    "<throws>": "throws",
    "<transient>": "transient",
    "<try>": "try",
    "<void>": "void",
    "<volatile>": "volatile",
    "<while>": "while",
}

# from https://docs.oracle.com/javase/tutorial/java/nutsandbolts/opsummary.html
java_operator_tokens = {
    "<=>": "=",
    "<+>": "+",
    "<->": "-",
    "<*>": "*",
    "</>": "/",
    "<%>": "%",
    "<++>": "++",
    "<-->": "--",
    "<!>": "!",
    "<==>": "==",
    "<!=>": "!=",
    "<greater>": ">",
    "<greater_equal>": ">=",
    "<lesser>": "<",
    "<lesser_equal>": "<=",
    "<&&>": "&&",
    "<||>": "||",
    "<?>": "?",
    "<:>": ":",
    "<~>": "~",
    "<double_lesser>": "<<",
    "<double_greater>": ">>",
    "<triple_greater>": ">>>",
    "<&>": "&",
    "<^>": "^",
    "<|>": "|",
}

java_structural_tokens = {
    "<{>": "{",
    "<}>": "}",
    "<[>": "[",
    "<]>": "]",
    "<lesser>": "<",
    "<greater>": ">",
    "<(>": "(",
    "<)>": ")",
    "<;>": ";",
}

java_extra_tokens = {
    "<@>": "@",
    "<...>": "...",
    "<null>": "null",
    "<true>": "true",
    "<false>": "false",
}

# combination of all dictionaries
java_special_tokens = {
    **java_reserved_tokens,
    **java_operator_tokens,
    **java_structural_tokens,
    **java_extra_tokens,
    **extra_tokens,
}

In [39]:
# hide
df_fake = pd.DataFrame([">>> > + public ++ \n\n \t \t \t\t  "], columns=["code"])
df_fake

,code
0,>>> > + public ++ \n\n \t \t \t\t


In [10]:
# export
def _replace_toks(mthd: str, spec_toks: Dict[str, str]) -> str:
    """
    Helper function for replacing all special tokens in a given method. This will replace longer special tokens first in order to not mistakenly breakup a special token that is part of a longer sequence. Adapted from https://stackoverflow.com/a/6117124/5768407 and https://stackoverflow.com/a/11753945/5768407

    :param mthd: the method to have its special tokens replaced
    :param spec_toks: a dictionary containing the special tokens to replace and the new tokens to replace them with
    :returns: returns the method with its special tokens replaced
    """
    # construct escaped versions of keys for running through regex
    spec_toks = dict(
        (re.escape(v), k)
        for k, v in sorted(
            java_special_tokens.items(), key=lambda x: len(x[1]), reverse=True
        )
    )
    # construct regex pattern for finding all special tokens in a method
    pattern = re.compile("|".join(spec_toks.keys()))
    # replace all special tokens in a method
    mthd = pattern.sub(lambda m: spec_toks[re.escape(m.group(0))], mthd)

    return mthd


def replace_special_tokens(
    df: pd.DataFrame, spec_toks: Dict[str, str], n: Optional[int] = None
) -> pd.DataFrame:
    """
    Replace all the special tokens in a pandas dataframe.

    :param df: the pandas dataframe containing each method to replace special tokens in
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :returns: returns a modified dataframe with the special tokens replaced
    """
    if n is None:
        n = len(df)

    df = df.iloc[:n].copy()
    df.code = df.code.apply(lambda mthd: _replace_toks(mthd, spec_toks))

    return df

In [24]:
REPLACED_MTHD = "<triple_greater> <greater> <+> <public> <++> <n><n> \t \t \t\t  "
df_replaced = replace_special_tokens(df_fake, java_special_tokens)

assert REPLACED_MTHD == df_replaced.code.values[0]

In [ ]:
# hide
fake_data = "<triple_greater> <greater> <+> <public> <++> <n><n>"

In [11]:
# export

def _replace_spec_toks(mthd: str, spec_toks: Dict[str, str]) -> str:
    """
    Performs the replacement of special tokens by the original ones
    """
    spec_toks = spec_toks.copy()
    # Add special tokenizer tokens -> deleted for code analysis
    spec_toks['<bos>'] = ""
    spec_toks['<eos>'] = ""
    spec_toks['<pad>'] = ""
    
    spec_toks = dict(
        (re.escape(k), v)
        for k, v in sorted(
            spec_toks.items(), key=lambda x: len(x[1]), reverse=True
        )
    )
    # construct regex pattern for finding all special tokens in a method
    pattern = re.compile("|".join(spec_toks.keys()))
    # replace all special tokens in a method
    mthd = pattern.sub(lambda m: spec_toks[re.escape(m.group(0))], mthd)
    
    mthd = __replace_tokenizer_toks(mthd)
    return mthd

def replace_spec_toks_to_original(df: pd.DataFrame, spec_toks: Dict[str, str],
                        n: Optional[int] = None) -> pd.DataFrame:
    if n is None:
        n = len(df)
    df = df.iloc[:n].copy()
    df.code = df.code.apply(lambda m: _replace_spec_toks(m, spec_toks))
    return df

In [29]:
df_fake = pd.DataFrame(["<triple_greater> <greater> <+> <public> <++> <n><n> \t \t \t\t  "], columns=["code"])

df_fake

,code
0,<triple_greater> <greater> <+> <public> <++> <...


In [30]:
df_replaced = replace_spec_toks_to_original(df_fake, java_special_tokens)
REPLACED_MTHD = ">>> > + public ++ \n\n \t \t \t\t  "

assert REPLACED_MTHD == df_replaced.code.values[0]

In [12]:
# export

def __replace_tokenizer_toks(code_snippet: str) -> str:
    """
    Function to replace special tokens introduced by the tokenizer/model (bos, eos, pad)
    :param code_snippet: String representing the code snippet
    :return: String containing the clean string
    """
    pattern = re.compile("|".join(["<pad>", "<sos>", "<eos>"]))
    
    clean_snippet = pattern.sub("", code_snippet)
    return clean_snippet

def replace_tokenizer_toks(df: pd.DataFrame, n: Optional[int]=None) -> pd.DataFrame:
    """
    Function to replreace
    :param df: Pandas DataFrame containing the collection of code snippets
    :return: Clean DataFrame
    """
    
    if n is None:
        n = len(df)
    df = df.iloc[:n].copy()
    df.code = df.code.apply(lambda snippet: __replace_tokenizer_toks(snippet))
    return df

In [ ]:
__replace_tokenizer_toks("<sos>public ahhhhhh () { }<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>")

'public ahhhhhh () { }'

In [ ]:
fake_snippet = "<sos>public methodName () { }<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><eos>"
fake_df = pd.DataFrame([fake_snippet], columns=["code"])
REPLACED_SNIPPET = "public methodName () { }"
replaced_df = replace_tokenizer_toks(fake_df)
assert REPLACED_SNIPPET == replaced_df.code.values[0]

In [13]:
# export
def train_tokenizer(
    df: pd.DataFrame,
    spec_toks: Dict[str, str],
    max_length: int,
    n: Optional[int] = None,
    vocab_sz: Optional[int] = 10_000,
    min_freq: Optional[int] = 2,
    output: Optional[Path] = None,
) -> Tokenizer:
    """
    Train a ByteLevel BPE tokenizer on a given pandas dataframe. Code adapted from https://github.com/huggingface/tokenizers/tree/master/bindings/python.

    :param df: the pandas dataframe containing each method to have the tokenizer train on
    :param spec_toks: dict of special tokens to add to the tokenizers so they do not get split
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :param vocab_sz: the maximum vocabulary size of the trained tokenizer. Defaulted was selected from: Big Code != Big Vocabulary: Open-Vocabulary Models for Source Code
    :param min_freq: the minimum frequency a token has to occur to be considered
    :returns: returns a trained ByteLevel BPE tokenizer
    """
    if n is None:
        n = len(df)

    # create tmp file to store df contents for training tokenizer
    tmp_path = Path("/tmp")
    tmp_path.mkdir(parents=True, exist_ok=True)
    with open(tmp_path / "tmp_tokenize.txt", "w") as f:
        f.write("\n".join(df.code.values[:n]))

    # initialize a tokenizer
    tokenizer = Tokenizer(models.BPE())

    # customize pre-tokenization and decoding
    tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
    tokenizer.decoder = decoders.ByteLevel()
    tokenizer.post_processor = processors.ByteLevel(trim_offsets=True)

    # train tokenizer with data in tmp file
    trainer = trainers.BpeTrainer(
        vocab_size=vocab_sz,
        min_frequency=min_freq,
        special_tokens=["<pad>", "<sos>", "<eos>"] + list(spec_toks.keys()),
    )
    tokenizer.train(trainer, [str(tmp_path / "tmp_tokenize.txt")])
    tokenizer.enable_padding(length=max_length, pad_token="<pad>")
    tokenizer.enable_truncation(max_length)

    # save tokenizer if output path given
    if output is not None:
        tokenizer.save(output, pretty=True)

    return tokenizer

In [ ]:
TOKENIZED_SPEC = [
    "<triple_greater>",
    "Ġ",
    "<greater>",
    "Ġ",
    "<+>",
    "Ġ",
    "<public>",
    "Ġ",
    "<++>",
    "Ġ",
    "<n>",
    "<n>",
    "<pad>",
    "<pad>",
    "<pad>",
    "<pad>",
]
max_length = 16
tokenizer = train_tokenizer(df_fake, java_special_tokens, max_length)
encoded = tokenizer.encode(fake_data)

assert TOKENIZED_SPEC == encoded.tokens

In [14]:
# export
def _split_input_target(mthd):
    input_text = mthd[:-1]
    target_text = mthd[1:]

    return input_text, target_text


def convert_df_to_tfds(
    df: pd.DataFrame, tokenizer: Tokenizer, max_length: int, batch_size: int
):
    tokenized_mthds = []
    for i in range(0, len(df.code.values), batch_size):
        batch = df.code.values[i : i + batch_size]
        batch = [f"<sos>{x}" for x in batch]
        for x in tokenizer.encode_batch(batch):
            tokenized_mthds.append(x.ids)

    ds = tf.data.Dataset.from_tensor_slices(tokenized_mthds)
    ds = ds.map(_split_input_target).batch(batch_size, drop_remainder=True)

    return ds

In [ ]:
batch_size = 1
dataset = convert_df_to_tfds(df_fake, tokenizer, max_length, batch_size)

for _, shape in dataset:
    assert (batch_size, max_length - 1) == shape.shape

In [19]:
# export

TYPES = 3
def _process_bigclone(path):
    for i in range(1, TYPES + 1):
        df = pd.read_json(path / f"bigclone-type-{i}.jsonl", orient="records", lines=True).rename(columns={"function_1": "code"})
        df = remove_non_ascii(df)
        
        control_df = df["code"].to_frame()
        control_df = replace_special_tokens(control_df, java_special_tokens)
        
        treatment_df = df["function_2"].to_frame().rename(columns={"function_2": "code"})
        treatment_df = replace_special_tokens(treatment_df, java_special_tokens)
        
        big_clone_df = pd.DataFrame(
            zip(control_df.code.values, treatment_df.code.values),
            columns=["function_1", "function_2"]
        )
        big_clone_df.to_json(path / f"bigclone-type-{i}.jsonl", orient="records", lines=True)

def _process_bug_fix(path):
    buggy_paths = sorted((path / "50-100").glob("buggy/*.java"))
    fixed_paths = sorted((path / "50-100").glob("fixed/*.java"))
    bugs = []
    fixes = []
    for bug_p, fix_p in zip(buggy_paths, fixed_paths):
        with open(bug_p, "r") as f:
            bugs.append(f.read())

        with open(fix_p, "r") as f:
            fixes.append(f.read())

    df_fix_bug = pd.DataFrame(zip(bugs, fixes), columns=["code", "fixes"])
    df_fix_bug = remove_non_ascii(df_fix_bug)
    df_buggy = pd.DataFrame(df_fix_bug.code.values, columns=["code"])
#     df_buggy = remove_non_ascii(df_buggy)
    df_buggy = replace_special_tokens(df_buggy, java_special_tokens)

    df_fixed = pd.DataFrame(df_fix_bug.fixes.values, columns=["code"])
#     df_fixed = remove_non_ascii(df_fixed)
    df_fixed = replace_special_tokens(df_fixed, java_special_tokens)

    # Saving to jsonl because csv formatting is causing issues with quoting
    df_buggy.to_json(path / "buggy.jsonl", orient="records", lines=True)
    df_fixed.to_json(path / "fixed.jsonl", orient="records", lines=True)


def _jsonl_list_to_dataframe(file_list, columns=None):
    """Load a list of jsonl.gz files into a pandas DataFrame."""
    return pd.concat(
        [
            pd.read_json(f, orient="records", compression="gzip", lines=True)[columns]
            for f in file_list
        ],
        sort=False,
    )


def _process_codesearchnet(path):
    """
    Grabs the different data splits and converts them into dataframes.
    Expects format from Code Search Net Challenge.
    """
    for split in ["train", "valid", "test"]:
        files = sorted((path / "java" / "final" / "jsonl" / split).glob("**/*.gz"))
        df = _jsonl_list_to_dataframe(files, ["code"])
        df = remove_non_ascii(df)
        df = replace_special_tokens(df, java_special_tokens)
        # Saving to jsonl because csv formatting is causing issues with quoting
        if split == "train":
            # 10% selected to match the Big Code != Big Vocab paper.
            df_trn, df_bpe = train_test_split(df, test_size=0.1)
            df_trn.to_json(path / f"{split}.jsonl", orient="records", lines=True)
            df_bpe.to_json(path / "bpe.jsonl", orient="records", lines=True)
        else:
            df.to_json(path / f"{split}.jsonl", orient="records", lines=True)


def _process_comment_testbed(path, out_path):
    df_tst = pd.read_json(path / "test.jsonl", orient="records", lines=True)
    df_replaced = replace_spec_toks_to_original(df_tst, java_special_tokens)
    df_no_cmts = transform_df(df_replaced, java_comment_remover)
    df_no_cmts = replace_special_tokens(df_no_cmts, java_special_tokens)
    
    # remove duplicates
    df_combined = pd.concat(
        [
            df_tst.rename(columns={"code": "commented"}), df_no_cmts.rename(columns={"code": "uncommented"})
        ], axis=1
    )
    non_dups = df_combined[df_combined.apply(lambda x: x["commented"] != x["uncommented"], axis = 1)]
    
    df_cmtd = pd.DataFrame(non_dups.commented.values, columns=["code"])
    df_uncmtd = pd.DataFrame(non_dups.uncommented.values, columns=["code"])
    
#     .code.update(non_dups.commented)
#     df_no_cmts.code.update(non_dups.uncommented)
#     df_no_cmts.code.values = non_dups.uncommented.values
    
    df_cmtd.to_json(out_path/"commented_code.jsonl", orient="records", lines=True)
    df_uncmtd.to_json(out_path/"uncommented_code.jsonl", orient="records", lines=True)
    

    
def process_data(path):
    """Function for processing data related to the library."""
    # Process Bug Fix Pairs data
    bugfix_path = path / "bug_fix"
    _process_bug_fix(bugfix_path)

    # Process CodeSearchNet Challenge data
    codesearchnet_path = path / "codesearchnet"
    _process_codesearchnet(codesearchnet_path / "codesearchnet_java")

In [20]:
path = Path("/home/jovyan/work/dvc-icodegen/testbed/ts-bigclone-types")
_process_bigclone(path)

In [69]:
path = Path("/home/jovyan/work/dvc-icodegen/datax/searchnet/codesearchnet-java")
out_path = Path("/home/jovyan/work/dvc-icodegen/testbed/ts-comments")
_process_comment_testbed(path, out_path)

In [78]:
df_tst = pd.read_json(path / "test.jsonl", orient="records", lines=True)
df_cmtd = pd.read_json(out_path / "commented_code.jsonl", orient="records", lines=True)
df_uncmtd = pd.read_json(out_path / "uncommented_code.jsonl", orient="records", lines=True)
len(df_tst), len(df_cmtd), len(df_uncmtd)

(26716, 6664, 6664)

In [ ]:
# bugfix_path = path / "bug_fix"
bugfix_path = Path("/home/jovyan/work/dvc-icodegen/bug_fix")
_process_bug_fix(bugfix_path)
df_buggy = pd.read_json(bugfix_path / "buggy.jsonl", orient="records", lines=True)
df_fixed = pd.read_json(bugfix_path / "fixed.jsonl", orient="records", lines=True)

In [ ]:
len(df_buggy), len(df_fixed)

(64722, 64722)

In [ ]:
len(df_buggy), len(df_fixed)

(65455, 65455)

In [ ]:
# export

def process_java_df(df: pd.DataFrame, n: Optional[int] = None) -> pd.DataFrame:
    """
    Performs preprocessing (rm non ascii and replacement of special chars.)
    :param df: Pandas Dataframe containing code (in 'code' column)
    :param n: Number of records to process
    :return: Processed pandas dataframe 
    """
    clean_df = remove_non_ascii(df, n)
    clean_df = replace_special_tokens(clean_df, java_special_tokens)
    
    return clean_df

In [ ]:
data_path = Path("/tmp/data/codesearchnet")
df_trn = pd.read_json(
    data_path / "codesearchnet_java" / "train.jsonl",
    orient="records",
    lines=True,
)[:50]
ds_trn = convert_df_to_tfds(df_trn, tokenizer, max_length, batch_size)

In [ ]:
# def huggingface_format(x, y):
#     print(x, y)
#     return {
#         "input_ids": x,
#         "labels": y
#     }
#     return x, y

# ds_trn = ds_trn.map(huggingface_format)

Tensor("args_0:0", shape=(1, 15), dtype=int32) Tensor("args_1:0", shape=(1, 15), dtype=int32)


In [ ]:
# for e in ds_trn:
#     print(e)

{'input_ids': <tf.Tensor: shape=(1, 15), dtype=int32, numpy=
array([[  1,  37, 102,  50, 102,  96,  96,  94,  99,  98,  96,  83,  98,
         96, 102]], dtype=int32)>, 'labels': <tf.Tensor: shape=(1, 15), dtype=int32, numpy=
array([[ 37, 102,  50, 102,  96,  96,  94,  99,  98,  96,  83,  98,  96,
        102,  98]], dtype=int32)>}
{'input_ids': <tf.Tensor: shape=(1, 15), dtype=int32, numpy=
array([[  1,  86,  96,  91, 100,  37, 102,   5, 102,  96,  97,  98,  96,
         83,  20]], dtype=int32)>, 'labels': <tf.Tensor: shape=(1, 15), dtype=int32, numpy=
array([[ 86,  96,  91, 100,  37, 102,   5, 102,  96,  97,  98,  96,  83,
         20, 102]], dtype=int32)>}
{'input_ids': <tf.Tensor: shape=(1, 15), dtype=int32, numpy=
array([[  1,  35, 102, 102,  98,  98,  83,  10, 102,  95,  96,  84, 102,
         79,  91]], dtype=int32)>, 'labels': <tf.Tensor: shape=(1, 15), dtype=int32, numpy=
array([[ 35, 102, 102,  98,  98,  83,  10, 102,  95,  96,  84, 102,  79,
         91, 102]], dtype=int32)>

In [ ]:
# from transformers import GPT2TokenizerFast, TFGPT2LMHeadModel
# from icodegen.model.core import _loss

# trnsfr_tokenizer = GPT2TokenizerFast.from_pretrained("sshleifer/tiny-gpt2")
# tokenizer = trnsfr_tokenizer.backend_tokenizer
# trnsfr = TFGPT2LMHeadModel.from_pretrained("sshleifer/tiny-gpt2");

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at sshleifer/tiny-gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
# trnsfr.compile(optimizer="adam", loss=trnsfr.compute_loss)
# history = trnsfr.fit(
#     ds_trn, epochs=1, batch_size=1
# )

ValueError: in user code:

    /opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:788 run_step  **
        outputs = model.t

In [ ]:
# Commenting out since depends on downloaded data
# process_data(data_path)

# assert Path(bugfix_path / "buggy.jsonl").exists()
# assert Path(bugfix_path / "fixed.jsonl").exists()

# assert Path(
#     codesearchnet_path / "codesearchnet_java" / "train.jsonl"
# ).exists()
# assert Path(
#     codesearchnet_path / "codesearchnet_java" / "bpe.jsonl"
# ).exists()
# assert Path(
#     codesearchnet_path / "codesearchnet_java" / "valid.jsonl"
# ).exists()
# assert Path(
#     codesearchnet_path / "codesearchnet_java" / "test.jsonl"
# ).exists()

In [ ]:
# BUGGY_MTHD = """\
# private void success(io.netty.channel.Channel channel) {
#     org.mycat.netty.mysql.MySQLHandshakeHandler.logger.debug("success info return form MySQLHandshakeHandler");
#     io.netty.buffer.ByteBuf out = channel.alloc().buffer();
#     org.mycat.netty.mysql.OK ok = new org.mycat.netty.mysql.OK();
#     ok.sequenceId = 2;
#     ok.setStatusFlag(Flags.SERVER_STATUS_AUTOCOMMIT);
#     out.writeBytes(ok.toPacket());
#     channel.writeAndFlush(out);
# }"""
# FIXED_MTHD = """\
# private void success(io.netty.channel.Channel channel) {
#     org.mycat.netty.mysql.MySQLHandshakeHandler.logger.info("success info return form MySQLHandshakeHandler");
#     io.netty.buffer.ByteBuf out = channel.alloc().buffer();
#     org.mycat.netty.mysql.OK ok = new org.mycat.netty.mysql.OK();
#     ok.sequenceId = 2;
#     ok.setStatusFlag(Flags.SERVER_STATUS_AUTOCOMMIT);
#     out.writeBytes(ok.toPacket());
#     channel.writeAndFlush(out);
# }"""
# df = pd.read_json(bugfix_path / "bug_fix_pairs.jsonl", orient="records", lines=True)

# assert BUGGY_MTHD == df.buggy.values[0] and FIXED_MTHD == df.fixed.values[0]

In [ ]:
# hide
from nbdev.export import notebook2script

notebook2script()

Converted 00_data.core.ipynb.
Converted 01_data.transforms.ipynb.
Converted 02_model.core.ipynb.
Converted 04_evaluation.core.ipynb.
Converted index.ipynb.
